# 02 - Preprocesamiento V2: Limpieza y Consolidación de la Serie

## 🧠 Fase 2: Refinamiento Estructural

Este notebook se encarga de transformar los datos crudos (descubiertos en la Fase 1) en una serie temporal pulida, continua y lista para el modelado. 

### 🎯 Objetivos Técnicos:
1. **Estandarización:** Renombrar columnas clave para simplicidad en el desarrollo.
2. **Tratamiento de Centinelas:** Identificar y normalizar valores no válidos.
3. **Garantía de Continuidad:** Asegurar frecuencia mensual MS sin huecos temporales.
4. **Imputación de Inercia:** Aplicar `fill forward` para completar la información faltante.
5. **Preservación de Anomalías:** Marcar outliers reales mediante variables binarias (dummies).
6. **Cumplimiento de Reglas de Negocio:** Truncar la serie para excluir el mes actual incompleto (Mes X).

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

print("✅ Entorno de preprocesamiento listo (V2).")

✅ Entorno de preprocesamiento listo (V2).


## 1. Carga de Datos y Estandarización de Nombres

Cargamos la serie desde `01_raw`. Siguiendo el análisis, cambiaremos la columna de ventas a un nombre más genérico y manejable.

In [2]:
RAW_DATA_PATH = "../data/01_raw/ventas_mensuales.csv"
df = pd.read_csv(RAW_DATA_PATH)
df['fecha'] = pd.to_datetime(df['fecha'])

# Regla 2: Renombrar unidades_vendidas_mensuales a unidades
df = df.rename(columns={'unidades_vendidas_mensuales': 'unidades'})

print(f"📊 Columnas actuales: {df.columns.tolist()}")
df.head()

📊 Columnas actuales: ['fecha', 'unidades']


,fecha,unidades
0,2018-12-01,29549
1,2019-01-01,21716
2,2019-02-01,12292
3,2019-03-01,15179
4,2019-04-01,11843


## 2. Tratamiento de Valores Centinela y Atípicos Reales

### 2.1 Centinelas a NaN
Siguiendo la **Regla 8**, cualquier valor identificado como ruido técnico (0, -1, 999) será convertido a NaN para luego ser imputado.

In [3]:
# Regla 8: Centinelas a NaN
sentinels = [0, -1, 999, 9999]
mask_sentinels = df['unidades'].isin(sentinels)
print(f"⚠️ Centinelas detectados y eliminados: {mask_sentinels.sum()}")
df.loc[mask_sentinels, 'unidades'] = np.nan

⚠️ Centinelas detectados y eliminados: 0


### 2.2 Etiquetado de Outliers Reales
Siguiendo la **Regla 3**, utilizaremos el método IQR para identificar valores inusuales pero reales, y crearemos una bandera binaria.

In [4]:
# Regla 3: Columna binaria para valores atípicos reales
Q1 = df['unidades'].quantile(0.25)
Q3 = df['unidades'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

df['es_atipico'] = ((df['unidades'] < lower_bound) | (df['unidades'] > upper_bound)).astype(int)
print(f"🚩 Outliers marcados como eventos binarios: {df['es_atipico'].sum()}")

🚩 Outliers marcados como eventos binarios: 5


## 3. Garantía de Continuidad Temporal y Regla del Mes X

### 3.1 Exclusión del Mes Actual (Incompleto)
Siguiendo la **Regla 6**, filtramos cualquier dato del mes actual para evitar sesgos por información parcial.

In [5]:
# Regla 6: Validar que el mes actual no haga parte de los datos
today = datetime.now()
first_day_current_month = datetime(today.year, today.month, 1)

mask_future = df['fecha'] >= first_day_current_month
if mask_future.any():
    print(f"🗑️ Eliminando datos del mes actual (incompleto): {df[mask_future]['fecha'].tolist()}")
    df = df[~mask_future]
else:
    print("✅ La serie no contiene el mes actual (está truncada correctamente).")

✅ La serie no contiene el mes actual (está truncada correctamente).


## 3.2 Frecuencia Mensual y Llenado de Huecos
Siguiendo las **Reglas 4, 6 y 7**, forzamos la serie a tener un registro por cada primer día de mes (MS) y completamos los nulos con Inercia (`fill forward`).

In [6]:
# Regla 4: Validar serie completa frecuencia MS
df = df.set_index('fecha').asfreq('MS')

missing_months = df['unidades'].isna().sum()
if missing_months > 0:
    print(f"🔍 Se detectaron {missing_months} huecos/nulos en la serie.")
    
    # Regla 7: Imputar utilizando fill forward
    df['unidades'] = df['unidades'].ffill()
    
    # Si el primer valor es NaN (no hay anterior), usamos bfill para estabilizar el arranque
    df['unidades'] = df['unidades'].bfill()
    
    print("✅ Huecos e inconsistencias imputados con Fill Forward.")
else:
    print("✅ La serie es continua y no presentó huecos.")

# Asegurar que la columna binaria también sea consistente tras asfreq
df['es_atipico'] = df['es_atipico'].fillna(0)

df = df.reset_index()

✅ La serie es continua y no presentó huecos.


## 4. Persistencia de Datos Limpios

Guardamos el resultado en la capa `02_cleansed` para que sea consumido por la fase de Ingeniería de Características.

In [7]:
OUTPUT_PATH = "../data/02_cleansed/ventas_preprocesadas.csv"
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

df.to_csv(OUTPUT_PATH, index=False)
print(f"💾 Serie preprocesada guardada exitosamente en: {OUTPUT_PATH}")
print(f"📏 Dimensiones finales: {df.shape}")

df.tail()

💾 Serie preprocesada guardada exitosamente en: ../data/02_cleansed/ventas_preprocesadas.csv
📏 Dimensiones finales: (83, 3)


,fecha,unidades,es_atipico
78,2025-06-01,189964.0,1
79,2025-07-01,200527.0,1
80,2025-08-01,167301.0,1
81,2025-09-01,138805.0,0
82,2025-10-01,148504.0,0
